# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [6]:
!az login -o table

In [7]:
!az account set --subscription $selected_subscription

In [10]:
!az account show

You will also need to register the container service resources on your subscription if you haven't already done so.

In [9]:
!az provider register -n Microsoft.ContainerService

In [10]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [11]:
!az group create --name $resource_group --location $location

Below, we create the AKS cluster in the resource group we created earlier.

In [12]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

{- Finished ..
  "additionalProperties": {},
  "agentPoolProfiles": [
    {
      "additionalProperties": {},
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-edf507",
  "fqdn": "fbaksclust-fbaksrg-edf507-97ac4d0f.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSCluster",
  "kubernetesVersion": "1.8.10",
  "linuxProfile": {
    "additionalProperties": {},
    "adminUsername": "azureuser",
    "ssh": {
      "additionalProperties": {},
      "publicKeys": [
        {
          "additionalProperties": {},
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCgTNSnvmSzdPGTzAvv8RkIm0wntQNwHEm0weIYisqfLFxYkfEAe2q5g

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [12]:
!sudo az aks install-cli

## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [14]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSCluster" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [15]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-28016997-0   Ready     agent     1d        v1.8.10


Let's check the pods on our cluster.

In [16]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   heapster-75f8df9884-bph6w               2/2       Running   0          1d
kube-system   kube-dns-v20-5bf84586f4-22snr           3/3       Running   0          1d
kube-system   kube-dns-v20-5bf84586f4-k6c72           3/3       Running   0          1d
kube-system   kube-proxy-bqtt8                        1/1       Running   0          1d
kube-system   kube-svc-redirect-4jqpq                 1/1       Running   2          1d
kube-system   kubernetes-dashboard-665f768455-mrmmk   1/1       Running   0          1d
kube-system   tunnelfront-76cbc478d6-l5z7j            1/1       Running   0          1d


## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [17]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/kerastf-gpu",
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "mountPath":"/usr/local/nvidia",
                              "name": "nvidia",
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [18]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [19]:
write_json_to_file(app_template, 'az-dl.json')

In [20]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [21]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                            }
                        ],
                        "image": "fboylu/kerastf-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
               

Next, we will use kubectl create command to deploy our application.

In [22]:
!kubectl create -f az-dl.json

deployment "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [23]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS              RESTARTS   AGE
default       azure-dl-5c4bfdb7f9-2cbr8               0/1       ContainerCreating   0          5s
kube-system   heapster-75f8df9884-bph6w               2/2       Running             0          1d
kube-system   kube-dns-v20-5bf84586f4-22snr           3/3       Running             0          1d
kube-system   kube-dns-v20-5bf84586f4-k6c72           3/3       Running             0          1d
kube-system   kube-proxy-bqtt8                        1/1       Running             0          1d
kube-system   kube-svc-redirect-4jqpq                 1/1       Running             2          1d
kube-system   kubernetes-dashboard-665f768455-mrmmk   1/1       Running             0          1d
kube-system   tunnelfront-76cbc478d6-l5z7j            1/1       Running             0          1d


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [38]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                  SOURCE                              MESSAGE
2m          2m           1         azure-dl-5c4bfdb7f9-2cbr8.1522f0360f3d02c1   Pod                                      Normal    Scheduled               default-scheduler                   Successfully assigned azure-dl-5c4bfdb7f9-2cbr8 to aks-nodepool1-28016997-0
2m          2m           1         azure-dl-5c4bfdb7f9-2cbr8.1522f03617f016e9   Pod                                      Normal    SuccessfulMountVolume   kubelet, aks-nodepool1-28016997-0   MountVolume.SetUp succeeded for volume "nvidia" 
2m          2m           1         azure-dl-5c4bfdb7f9-2cbr8.1522f03618d2762c   Pod                                      Normal    SuccessfulMountVolume   kubelet, aks-nodepool1-28016997-0   MountVolume.SetUp succeeded for volume "default-token-m6g6j" 
2m          2m           1         azure-

Check the logs for the application pod.

In [34]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [41]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-04-06 19:44:14,983 CRIT Supervisor running as root (no user in config file)
2018-04-06 19:44:14,985 INFO supervisord started with pid 1
2018-04-06 19:44:15,988 INFO spawned: 'program_exit' with pid 11
2018-04-06 19:44:15,989 INFO spawned: 'nginx' with pid 12
2018-04-06 19:44:15,990 INFO spawned: 'gunicorn' with pid 13
2018-04-06 19:44:17,021 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-04-06 19:44:17.421986: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-04-06 19:44:21,427 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
2018-04-06 19:44:24.311778: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: f115:00:00.0
totalMemory: 11.17GiB freeMe

In [40]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   1         1         1            1           2m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [42]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP    EXTERNAL-IP    PORT(S)        AGE
azure-dl   LoadBalancer   10.0.12.158   52.186.67.95   80:30022/TCP   2m


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources.

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [70]:
!kubectl delete -f az-dl.json

deployment "azure-dl" deleted
service "azure-dl" deleted


In [31]:
!az aks delete -n $aks_name -g $resource_group -y

inished ..

In [32]:
!az group delete --name $resource_group -y

inished ..